## Importing Libraries

In [31]:
# Import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import essential models and functions from sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import plot_tree

## Importing Data Set

In [32]:
dataset = pd.read_csv('dataset/pca-sleep-efficiency.csv')
dataset.head()

,id,age,gender,bed_time,wakeup_time,sleep_duration,sleep_efficiency,rem_sleep_percentage,deep_sleep_percentage,light_sleep_percentage,...,smoking_status,exercise_frequency,exercise_frequency_filled,bed_time_edited,bed_time_encoded,wakeup_time_edited,wakeup_time_encoded,smoking,gender_type,sleep_quality
0,1,65,Female,01:00,07:00,6.0,0.88,18,70,12,...,Yes,3.0,3.0,1,1.0,7,7.0,1,0,-1.038628
1,2,69,Male,02:00,09:00,7.0,0.66,19,28,53,...,Yes,3.0,3.0,2,2.0,9,9.0,1,1,1.705723
2,3,40,Female,21:30,05:30,8.0,0.89,20,70,10,...,No,3.0,3.0,21,21.0,5,5.0,0,0,-0.876008
3,4,40,Female,02:30,08:30,6.0,0.51,23,25,52,...,Yes,1.0,1.0,2,2.0,8,8.0,1,0,2.028781
4,5,57,Male,01:00,09:00,8.0,0.76,27,55,18,...,No,3.0,3.0,1,1.0,9,9.0,0,1,0.192627


## Train Test Split

In [33]:
X = dataset[['age','gender_type','wakeup_time_encoded','bed_time_encoded','sleep_duration', 'exercise_frequency_filled', 'caffeine_consumption_filled', 'alcohol_consumption_filled', 'smoking']]
y = dataset['sleep_quality']

# Perform first split
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Perform the second split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=100)

In [34]:
# Create the scaler with object range of 0-1
scaler = MinMaxScaler()

# Fit and transform using the training data
scaler.fit_transform(X_train)

# Transform the validation and test features
scaler.transform(X_valid)
scaler.transform(X_test)

array([[0.37288136, 0.        , 0.55555556, 0.04347826, 0.4       ,
        0.        , 0.125     , 0.2       , 0.        ],
       [0.3220339 , 0.        , 0.55555556, 0.        , 0.6       ,
        0.2       , 0.125     , 0.8       , 0.        ],
       [0.52542373, 0.        , 0.22222222, 0.91304348, 0.5       ,
        0.6       , 0.375     , 0.4       , 1.        ],
       [0.33898305, 0.        , 0.22222222, 0.        , 0.        ,
        0.4       , 0.375     , 0.        , 1.        ],
       [0.15254237, 1.        , 0.66666667, 0.04347826, 0.5       ,
        0.6       , 0.        , 0.        , 0.        ],
       [0.25423729, 0.        , 0.22222222, 0.95652174, 0.4       ,
        0.        , 0.125     , 0.        , 0.        ],
       [0.66101695, 0.        , 0.22222222, 0.95652174, 0.4       ,
        0.8       , 0.        , 0.4       , 0.        ],
       [0.54237288, 1.        , 0.22222222, 0.95652174, 0.4       ,
        0.6       , 0.        , 0.4       , 1.        ],


In [35]:
# Create a list of the columns to drop
cols_to_drop = ['gender_type', 'wakeup_time_encoded', 'sleep_duration','caffeine_consumption_filled','smoking']

# Drop these columns from training, validation and test data
X_train_temp.drop(columns=cols_to_drop,inplace=True)
X_train.drop(columns=cols_to_drop,inplace=True)
X_valid.drop(columns=cols_to_drop,inplace=True)
X_test.drop(columns=cols_to_drop,inplace=True)

## Random Forest Regression

In [36]:
# Create regressor
rf_regressor = RandomForestRegressor(random_state=100)

# Fit RF model to training data
rf = rf_regressor.fit(X_train, y_train)

## Evaluate Model Performance

In [37]:
def scoring(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mae = mean_absolute_error(test_labels, predictions)
    mse = mean_squared_error(test_labels, predictions)
    r2 = r2_score(test_labels, predictions)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - abs(mape)
    print('Model Performance')
    print('Mean Absolute Error: {:0.4f}.'.format(mae))
    print('Mean Squared Error: {:0.4f}.'.format(mse))
    print('R^2 Score = {:0.4f}.'.format(r2))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

In [38]:
# Evaluate the performance
scoring(rf, X_valid, y_valid)

Model Performance
Mean Absolute Error: 0.7624.
Mean Squared Error: 0.9725.
R^2 Score = 0.2116.
Accuracy = 87.47%.


r2_score (also known as the coefficient of determination) measures the proportion of variance in the response variable that is explained by the predictors in the model. It is defined as 1 minus the ratio of the residual variance to the total variance of the response variable. The r2_score ranges from -1 to 1, with higher values indicating a better fit of the model to the data. A value of 1 indicates a perfect fit, while a value of 0 indicates that the model does not explain any of the variability in the response variable.

### Feature Importance

In [39]:
# Get feature importances
importances = rf.feature_importances_
for i, importance in enumerate(importances):
    print(f'Feature {i+1}: {X.columns[i]} - Importance: {importance}')

Feature 1: age - Importance: 0.4282832735692112
Feature 2: gender_type - Importance: 0.14509680551592458
Feature 3: wakeup_time_encoded - Importance: 0.17546612114015542
Feature 4: bed_time_encoded - Importance: 0.25115379977470886


In feature importance, the values represent the importance of each feature in predicting the target variable. It is typically used to identify the most important features in a dataset that are relevant for predicting the target variable. The higher the value, the more important the feature is. 

This is the ranking of the features according to their importance, from the above computation:
| Rank | Features | Feature importance |
| ------------ | --------- | ------------|
| 1 | age | 0.25287546869016014 |
| 2 | alcohol_consumption| 0.20639532408183847 |
| 3 | exercise_frequency | 0.13181440737931946 |
| 4 | smoking | 0.10328762844484661 |
| 5 | wakeup_time_encoded | 0.09436161480691739 |
| 6 | sleep_duration | 0.083304773803437 |
| 7 | caffeine_consumption | 0.05284697946805141 |
| 8 | bed_time_encoded | 0.051652960648978584 |
| 9 | gender_type | 0.023460842676450943 |

Age, alcohol consumption, and exercise frequency are the most important features while features like bedtime, and gender were less relevant in predicting our target variable – sleep quality.

### To change because we dropped variables

### Make Prediction of Sleep Quality based on model(to change)

In [40]:
# Make a prediction for new data
new_data = pd.DataFrame({'age': [50], 'bed_time_encoded': [22], 'exercise_frequency_filled': [0], 'alcohol_consumption_filled': [1]})
prediction = rf.predict(new_data)
print('Prediction:', prediction)

Prediction: [0.59377431]
